# Page Generator Agent

This notebook generates new Angular master pages using the component metadata collected by the component_metadata_generator.

## Pipeline:
1. Load component metadata (available components with their code)
2. Take user request for a new page (e.g., "Sample Program")
3. Generate HTML, SCSS, and TypeScript files using LLM (with available components)
4. Create a new directory in master folder
5. Save the generated files
6. Update master.module.ts with import, route, and declaration

## Step 1: Import Required Libraries

In [1]:
import os
import json
import re
from pathlib import Path
from get_secrets import run_model

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Step 2: Load Component Metadata

Load the metadata generated by component_metadata_generator

In [2]:
# Load component metadata
metadata_file = Path("component_metadata.json")

if metadata_file.exists():
    with open(metadata_file, 'r', encoding='utf-8') as f:
        component_metadata = json.load(f)
    print(f"✓ Loaded metadata for {len(component_metadata)} components")
    for comp in component_metadata:
        print(f"  - {comp['name']} (id: {comp['id_name']})")
else:
    print("❌ component_metadata.json not found!")
    print("Please run the component_metadata_generator notebook first.")
    component_metadata = []

✓ Loaded metadata for 4 components
  - AppButtonComponent (id: app-button)
  - AppDataTableComponent (id: app-data-table)
  - AppFooterComponent (id: app-footer)
  - AppHeaderComponent (id: app-header)


## Step 3: Configure Paths and Helper Functions

In [3]:
# Define master directory path
MASTER_DIR = Path(r"C:\Users\ManasSanjayPakalapat\Documents\cloudangles\motherson_demo\src\app\master")
MASTER_MODULE_FILE = MASTER_DIR / "master.module.ts"

print(f"Master directory: {MASTER_DIR}")
print(f"Master module file: {MASTER_MODULE_FILE}")
print(f"Master directory exists: {MASTER_DIR.exists()}")

def to_kebab_case(text):
    """Convert text to kebab-case (e.g., 'Sample Program' -> 'sample-program')"""
    # Replace spaces and underscores with hyphens, convert to lowercase
    text = re.sub(r'[_\s]+', '-', text)
    text = re.sub(r'([a-z0-9])([A-Z])', r'\1-\2', text)
    return text.lower().strip('-')

def to_pascal_case(text):
    """Convert text to PascalCase (e.g., 'sample program' -> 'SampleProgram')"""
    words = re.split(r'[_\s-]+', text)
    return ''.join(word.capitalize() for word in words if word)

# Test the helper functions
test_name = "Sample Program"
print(f"\nTest conversions for '{test_name}':")
print(f"  Kebab-case: {to_kebab_case(test_name)}")
print(f"  PascalCase: {to_pascal_case(test_name)}")

Master directory: C:\Users\ManasSanjayPakalapat\Documents\cloudangles\motherson_demo\src\app\master
Master module file: C:\Users\ManasSanjayPakalapat\Documents\cloudangles\motherson_demo\src\app\master\master.module.ts
Master directory exists: True

Test conversions for 'Sample Program':
  Kebab-case: sample-program
  PascalCase: SampleProgram


## Step 4: Create System Prompt for Page Generation

This prompt instructs the LLM to generate complete Angular page files using available components.

In [4]:
def create_page_generation_prompt(component_metadata):
    """
    Create a comprehensive system prompt for page generation
    """
    
    # Build components documentation
    components_doc = "Available Angular Components:\n\n"
    for comp in component_metadata:
        components_doc += f"Component: {comp['name']}\n"
        components_doc += f"Description: {comp['description']}\n"
        components_doc += f"HTML Tag/ID to use: {comp['id_name']}\n"
        components_doc += f"Import Path: {comp['import_path']}\n"
        components_doc += f"---\n\n"
    
    system_prompt = f"""You are an expert Angular developer creating new master pages.

You will be given a page requirement and you must generate THREE files: HTML, SCSS, and TypeScript.

{components_doc}

IMPORTANT RULES:
1. Use the available components listed above in your HTML(If they are not violating the user preference, alaways try to use them)
2. For buttons, use: <app-button>Click Me</app-button>
3. For header, use: <app-header></app-header>
4. For footer, use: <app-footer></app-footer>
5. The TypeScript file MUST:
   - Import Component from '@angular/core'
   - Have proper @Component decorator with selector, templateUrl, styleUrls
   - Export the component class
   - Include OnInit lifecycle hook
   - Have proper constructor and ngOnInit method

EXAMPLE HTML:
```html
<div class="page-container">
  <app-header></app-header>
  
  <div class="content">
    <h1>My Page Title</h1>
    <app-button>Submit</app-button>
  </div>
  
  <app-footer></app-footer>
</div>
```

EXAMPLE SCSS:
```scss
.page-container {{
  display: flex;
  flex-direction: column;
  min-height: 100vh;
}}

.content {{
  flex: 1;
  padding: 20px;
}}
```

EXAMPLE TypeScript:
```typescript
import {{ Component, OnInit }} from '@angular/core';

@Component({{
  selector: 'app-sample-program',
  templateUrl: './sample-program.component.html',
  styleUrls: ['./sample-program.component.scss']
}})
export class SampleProgramComponent implements OnInit {{

  constructor() {{ }}

  ngOnInit(): void {{
    // Initialization logic here
  }}

}}
```

You MUST return ONLY a valid JSON object with this exact structure:
{{
  "component_name": "SampleProgramComponent",
  "path_name": "sample-program",
  "selector": "app-sample-program",
  "html_code": "complete HTML code here",
  "scss_code": "complete SCSS code here",
  "ts_code": "complete TypeScript code here"
}}

Rules for naming:
- component_name: PascalCase with "Component" suffix (e.g., "SampleProgramComponent")
- path_name: kebab-case (e.g., "sample-program")
- selector: "app-" + path_name (e.g., "app-sample-program")

Return ONLY the JSON object, no additional text or markdown."""

    return system_prompt

SYSTEM_PROMPT = create_page_generation_prompt(component_metadata)
print("✓ System prompt created with component metadata")

✓ System prompt created with component metadata


## Step 5: Generate Page Files with LLM

Call the LLM to generate HTML, SCSS, and TypeScript code for the new page.

In [5]:
async def generate_page_code(page_description):
    """
    Generate page files using LLM
    
    Args:
        page_description: User's description of the page to create
        
    Returns:
        dict: Generated code and metadata
    """
    print(f"\n{'='*60}")
    print(f"GENERATING PAGE: {page_description}")
    print(f"{'='*60}\n")
    
    user_message = f"""Create a new Angular master page for: {page_description}

Please generate a complete Angular component with HTML, SCSS, and TypeScript files.
Use the available components (app-header, app-footer, app-button, etc.) appropriately.

The page should be well-structured, professional, and follow Angular best practices."""

    print("⏳ Calling LLM to generate page code...")
    
    try:
        response = await run_model(
            system_prompt=SYSTEM_PROMPT,
            user_message=user_message
        )
        
        print("✓ Received response from LLM")
        
        # Parse JSON response
        response_text = response.strip()
        
        # Remove markdown code blocks if present
        if response_text.startswith("```"):
            lines = response_text.split('\n')
            # Find the JSON content between code fences
            json_start = 1
            json_end = len(lines) - 1
            for i, line in enumerate(lines):
                if i > 0 and line.strip() and not line.strip().startswith('```'):
                    json_start = i
                    break
            for i in range(len(lines) - 1, 0, -1):
                if lines[i].strip().startswith('```'):
                    json_end = i
                    break
            response_text = '\n'.join(lines[json_start:json_end])
        
        page_data = json.loads(response_text)
        
        print(f"\n✓ Successfully parsed page data:")
        print(f"  Component Name: {page_data.get('component_name')}")
        print(f"  Path Name: {page_data.get('path_name')}")
        print(f"  Selector: {page_data.get('selector')}")
        print(f"  HTML Code: {len(page_data.get('html_code', ''))} characters")
        print(f"  SCSS Code: {len(page_data.get('scss_code', ''))} characters")
        print(f"  TS Code: {len(page_data.get('ts_code', ''))} characters")
        
        return page_data
        
    except json.JSONDecodeError as e:
        print(f"❌ Error parsing JSON response: {e}")
        print(f"Response was: {response[:500]}...")
        return None
    except Exception as e:
        print(f"❌ Error generating page: {e}")
        return None

# Example: Generate a page
# Change this to your desired page description
PAGE_DESCRIPTION = "Create a simple welcome page. Welcome Text should mention Are you Proud to be part of, with a button saying MotherSon, where clicking it should imply that you will be going into the next page."

# generated_page = await generate_page_code(PAGE_DESCRIPTION)

## Step 6: Create Directory and Save Files

Create the component directory and save all generated files.

In [7]:
def save_page_files(page_data):
    """
    Create directory and save all generated files
    
    Args:
        page_data: Dictionary containing generated code and metadata
        
    Returns:
        Path: Path to the created directory
    """
    if not page_data:
        print("❌ No page data to save")
        return None
    
    path_name = page_data['path_name']
    component_dir = MASTER_DIR / path_name
    
    print(f"\n{'='*60}")
    print(f"SAVING FILES")
    print(f"{'='*60}\n")
    
    # Create directory
    print(f"Creating directory: {component_dir}")
    component_dir.mkdir(parents=True, exist_ok=True)
    print("✓ Directory created")
    
    # Save HTML file
    html_file = component_dir / f"{path_name}.component.html"
    print(f"\nSaving: {html_file.name}")
    with open(html_file, 'w', encoding='utf-8') as f:
        f.write(page_data['html_code'])
    print(f"✓ Saved ({len(page_data['html_code'])} chars)")
    
    # Save SCSS file
    scss_file = component_dir / f"{path_name}.component.scss"
    print(f"\nSaving: {scss_file.name}")
    with open(scss_file, 'w', encoding='utf-8') as f:
        f.write(page_data['scss_code'])
    print(f"✓ Saved ({len(page_data['scss_code'])} chars)")
    
    # Save TypeScript file
    ts_file = component_dir / f"{path_name}.component.ts"
    print(f"\nSaving: {ts_file.name}")
    with open(ts_file, 'w', encoding='utf-8') as f:
        f.write(page_data['ts_code'])
    print(f"✓ Saved ({len(page_data['ts_code'])} chars)")
    
    # Create spec file (basic template)
    spec_file = component_dir / f"{path_name}.component.spec.ts"
    print(f"\nCreating: {spec_file.name}")
    spec_content = f"""import {{ ComponentFixture, TestBed }} from '@angular/core/testing';

import {{ {page_data['component_name']} }} from './{path_name}.component';

describe('{page_data['component_name']}', () => {{
  let component: {page_data['component_name']};
  let fixture: ComponentFixture<{page_data['component_name']}>;

  beforeEach(async () => {{
    await TestBed.configureTestingModule({{
      declarations: [ {page_data['component_name']} ]
    }})
    .compileComponents();
  }});

  beforeEach(() => {{
    fixture = TestBed.createComponent({page_data['component_name']});
    component = fixture.componentInstance;
    fixture.detectChanges();
  }});

  it('should create', () => {{
    expect(component).toBeTruthy();
  }});
}});
"""
    with open(spec_file, 'w', encoding='utf-8') as f:
        f.write(spec_content)
    print(f"✓ Saved spec file")
    
    print(f"\n{'='*60}")
    print(f"ALL FILES SAVED SUCCESSFULLY")
    print(f"Location: {component_dir}")
    print(f"{'='*60}\n")
    
    return component_dir

# if generated_page:
#     component_directory = save_page_files(generated_page)

## Step 7: Update master.module.ts

Add the import statement, route, and declaration to master.module.ts

In [9]:
def update_master_module(page_data):
    """
    Update master.module.ts with import, route, and declaration
    
    Args:
        page_data: Dictionary containing component metadata
    """
    if not page_data:
        print("❌ No page data to update module")
        return
    
    print(f"\n{'='*60}")
    print(f"UPDATING master.module.ts")
    print(f"{'='*60}\n")
    
    component_name = page_data['component_name']
    path_name = page_data['path_name']
    
    # Read the current module file
    with open(MASTER_MODULE_FILE, 'r', encoding='utf-8') as f:
        module_content = f.read()
    
    print("✓ Read master.module.ts")
    
    # 1. Add import statement
    import_statement = f"import {{ {component_name} }} from './{path_name}/{path_name}.component';"
    
    # Check if import already exists
    if import_statement in module_content:
        print(f"⚠ Import for {component_name} already exists")
    else:
        # Find the last import statement and add after it
        import_lines = [line for line in module_content.split('\n') if line.strip().startswith('import ')]
        if import_lines:
            last_import = import_lines[-1]
            module_content = module_content.replace(last_import, f"{last_import}\n{import_statement}")
            print(f"✓ Added import: {component_name}")
        else:
            print("❌ Could not find import section")
    
    # 2. Add route
    route_entry = f"  {{ path: '{path_name}', component: {component_name} }}"
    
    # Find the routes array and add the new route
    if route_entry not in module_content:
        # Find the routes array
        routes_match = re.search(r'const routes = \[(.*?)\];', module_content, re.DOTALL)
        if routes_match:
            current_routes = routes_match.group(1)
            # Add new route before the closing bracket
            new_routes = current_routes.rstrip() + ",\n" + route_entry + "\n"
            module_content = module_content.replace(
                f"const routes = [{current_routes}];",
                f"const routes = [{new_routes}];"
            )
            print(f"✓ Added route: {path_name}")
        else:
            print("❌ Could not find routes array")
    else:
        print(f"⚠ Route for {path_name} already exists")
    
    # 3. Add to declarations
    if component_name not in module_content.split('declarations: [')[1].split(']')[0]:
        # Find the declarations array
        declarations_match = re.search(r'declarations: \[(.*?)\]', module_content, re.DOTALL)
        if declarations_match:
            current_declarations = declarations_match.group(1)
            # Add new component to declarations
            new_declarations = current_declarations.rstrip() + ",\n    " + component_name + "\n  "
            module_content = module_content.replace(
                f"declarations: [{current_declarations}]",
                f"declarations: [{new_declarations}]"
            )
            print(f"✓ Added to declarations: {component_name}")
        else:
            print("❌ Could not find declarations array")
    else:
        print(f"⚠ {component_name} already in declarations")
    
    # Write the updated content back
    with open(MASTER_MODULE_FILE, 'w', encoding='utf-8') as f:
        f.write(module_content)
    
    print(f"\n{'='*60}")
    print(f"master.module.ts UPDATED SUCCESSFULLY")
    print(f"{'='*60}\n")
    
    return True

# if generated_page:
#     update_master_module(generated_page)

## Step 8: Complete Pipeline - Generate Any Page

This is the main function that runs the complete pipeline.

In [10]:
async def generate_new_page(page_description):
    """
    Complete pipeline to generate a new Angular master page
    
    Args:
        page_description: Description of the page to create
        
    Returns:
        bool: True if successful, False otherwise
    """
    print(f"\n{'#'*60}")
    print(f"STARTING PAGE GENERATION PIPELINE")
    print(f"Page: {page_description}")
    print(f"{'#'*60}\n")
    
    # Step 1: Generate code with LLM
    page_data = await generate_page_code(page_description)
    if not page_data:
        print("❌ Failed to generate page code")
        return False
    
    # Step 2: Save files to directory
    component_dir = save_page_files(page_data)
    if not component_dir:
        print("❌ Failed to save files")
        return False
    
    # Step 3: Update master.module.ts
    success = update_master_module(page_data)
    if not success:
        print("❌ Failed to update master.module.ts")
        return False
    
    print(f"\n{'#'*60}")
    print(f"PAGE GENERATION COMPLETE!")
    print(f"{'#'*60}\n")
    print(f"✓ Component created: {page_data['component_name']}")
    print(f"✓ Location: {component_dir}")
    print(f"✓ Route: /{page_data['path_name']}")
    print(f"✓ Module updated: master.module.ts")
    
    return True

# Example usage - change this to generate different pages
# await generate_new_page("Sample Program")
# await generate_new_page("User Registration Form")
# await generate_new_page("Dashboard Overview")

In [12]:
User_task = """
Can you generate a new page that displays a list of users in a table format with columns for Name, Email, and Role with some data on your own.
"""
await generate_new_page(User_task)


############################################################
STARTING PAGE GENERATION PIPELINE
Page: 
Can you generate a new page that displays a list of users in a table format with columns for Name, Email, and Role with some data on your own.

############################################################


GENERATING PAGE: 
Can you generate a new page that displays a list of users in a table format with columns for Name, Email, and Role with some data on your own.


⏳ Calling LLM to generate page code...
✓ Received response from LLM

✓ Successfully parsed page data:
  Component Name: UserListComponent
  Path Name: user-list
  Selector: app-user-list
  HTML Code: 899 characters
  SCSS Code: 1234 characters
  TS Code: 3627 characters

SAVING FILES

Creating directory: C:\Users\ManasSanjayPakalapat\Documents\cloudangles\motherson_demo\src\app\master\user-list
✓ Directory created

Saving: user-list.component.html
✓ Saved (899 chars)

Saving: user-list.component.scss
✓ Saved (1234 chars)


True